<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/02_cuda_lab/00_UnifiedMemory_SharedMem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Unified Memory Test & Shared Memory vs Global Memory Test

# Unified Memory Test

참조: https://devblogs.nvidia.com/unified-memory-cuda-beginners/


cudaMalloc 및 cudaMemCpy 등에 대한 고려 없이 마치 host에서 메모리를 사용하 듯이 GPU의 메모리를 사용함

**cudaMallocManaged** 를 사용합니다!!!



In [1]:
%%writefile unifiedMem.cu

#include <iostream>
#include <math.h>
 
// CUDA kernel to add elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}
 
int main(void)
{
  int N = 1<<20;
  float *x, *y;
 
  // Allocate Unified Memory -- accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
 
  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }
 
  // Launch kernel on 1M elements on the GPU
  int blockSize = 256;
  int numBlocks = (N + blockSize - 1) / blockSize;
  add<<<numBlocks, blockSize>>>(N, x, y);
 
  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();
 
  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;
 
  // Free memory
  cudaFree(x);
  cudaFree(y);
 
  return 0;
}

Writing unifiedMem.cu


In [0]:
!nvcc -o unifiedMem unifiedMem.cu

In [3]:
!./unifiedMem

Max error: 0


In [4]:
!nvprof ./unifiedMem

==404== NVPROF is profiling process 404, command: ./unifiedMem
Max error: 0
==404== Profiling application: ./unifiedMem
==404== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  102.46us         1  102.46us  102.46us  102.46us  add(int, float*, float*)
      API calls:   98.33%  219.67ms         2  109.84ms  803.66us  218.87ms  cudaMallocManaged
                    0.83%  1.8441ms         1  1.8441ms  1.8441ms  1.8441ms  cudaLaunchKernel
                    0.52%  1.1627ms         2  581.33us  486.14us  676.53us  cudaFree
                    0.17%  379.82us        96  3.9560us     157ns  150.05us  cuDeviceGetAttribute
                    0.08%  189.25us         1  189.25us  189.25us  189.25us  cuDeviceTotalMem
                    0.05%  116.81us         1  116.81us  116.81us  116.81us  cudaDeviceSynchronize
                    0.01%  23.703us         1  23.703us  23.703us  23.703us  cuDeviceGetName
           

- 참조: 
    - https://devblogs.nvidia.com/unified-memory-cuda-beginners/
    - https://devblogs.nvidia.com/maximizing-unified-memory-performance-cuda/

#  Shared Memory vs Global Memory Test
